## Dependencies


In [1]:
import torch
import os
from PIL import Image
from pathlib import Path
import pandas as pd
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
os.chdir('../')
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
from src.classifier1 import ResNet50Classifier
from src.train import train_classifier

In [2]:
%pwd

'c:\\Projects\\python\\brain_tumor_segmentation'

In [3]:
train_path = Path('.\data\\brain-tumor-mri-dataset\Training')
test_path = Path('.\data\\brain-tumor-mri-dataset\Testing')
H,W = 256,256
torch.manual_seed(42)
batch_size = 16
lr = 1e-3
num_epochs = 50
model_dir = Path('./models/classifier1/')
log_path = Path('./logs/train_log_resnet50_1.csv')
os.makedirs(model_dir,exist_ok=True)
model_path = os.path.join(model_dir, 'best_model.pt')
device='cuda'

## Data loader

In [4]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()
])

train_dataset = ImageFolder(root=train_path,transform=transform)
valid_size = int(len(train_dataset)*0.15)
train_size = len(train_dataset)-valid_size
test_data = ImageFolder(root=test_path,transform=transform)
train_data,valid_data = random_split(train_dataset,[train_size,valid_size])

train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
valid_loader = DataLoader(dataset=valid_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32,shuffle=True)

## Train

In [5]:
model = ResNet50Classifier(num_classes=4)  #changed

train_classifier(model=model, train_loader=train_loader, valid_loader=valid_loader, device=device, num_epochs=num_epochs, lr=lr,
                 log_path=log_path, model_path=model_path, patience=5)  #changed


c:\Projects\python\mainenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Projects\python\mainenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50, Train Loss: 0.4265, Train Acc: 0.8478, Valid Loss: 0.5495, Valid Acc: 0.8236, Patience: 0
Epoch 2/50, Train Loss: 0.2575, Train Acc: 0.9117, Valid Loss: 0.3365, Valid Acc: 0.8773, Patience: 0
Epoch 3/50, Train Loss: 0.2240, Train Acc: 0.9195, Valid Loss: 0.2816, Valid Acc: 0.9089, Patience: 0
Epoch 4/50, Train Loss: 0.1506, Train Acc: 0.9514, Valid Loss: 0.2687, Valid Acc: 0.9194, Patience: 0
Epoch 5/50, Train Loss: 0.1515, Train Acc: 0.9463, Valid Loss: 1.2879, Valid Acc: 0.7220, Patience: 0
Epoch 6/50, Train Loss: 0.1301, Train Acc: 0.9578, Valid Loss: 0.2807, Valid Acc: 0.8843, Patience: 1
Epoch 7/50, Train Loss: 0.1108, Train Acc: 0.9646, Valid Loss: 0.2821, Valid Acc: 0.9112, Patience: 2
Epoch 8/50, Train Loss: 0.0704, Train Acc: 0.9745, Valid Loss: 0.1365, Valid Acc: 0.9568, Patience: 3
Epoch 9/50, Train Loss: 0.0719, Train Acc: 0.9751, Valid Loss: 0.2417, Valid Acc: 0.9112, Patience: 0
Epoch 10/50, Train Loss: 0.0768, Train Acc: 0.9722, Valid Loss: 0.3689, Valid Acc:

## Test

In [6]:
# Load the best saved model weights
model.load_state_dict(torch.load(model_path))  #changed
model.eval()

correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  #changed
        outputs = model(images)  #changed
        _, predicted = torch.max(outputs, 1)  #changed
        correct_test += (predicted == labels).sum().item()  #changed
        total_test += labels.size(0)  #changed

print("Test Accuracy: {:.4f}".format(correct_test / total_test))

Test Accuracy: 0.9672
